<center><a href="https://5loi.com/about_loi"><img src="images/DLI_Header.png" width="400" height="186" /></a></center>

# 6. 自然语言处理

在这个教程中，我们将离开像静态图像这样的独立数据，转向依赖于序列中其它数据项的数据。作为示例，我们将使用句子文本。语言自然由序列数据组成，形式为单词中的字符和句子中的单词。其它序列数据的例子包括股票价格和天气数据等时间序列。视频虽然包含静态图像，但也属于序列。当数据中的元素与之前和之后的内容有关系，我们就需要采取不同的方法来处理。

## 6.1 目标

* 使用分词器为神经网络准备文本
* 观察如何使用嵌入来识别文本数据的数值特征

## 6.2 BERT 

BERT，全称为 **B**idirectional **E**ncoder **R**epresentations from **T**ransformers，是 [Google](https://www.google.com/) 在 2018 年引入的一种开创性模型。

BERT 同时针对两个目标进行训练：
* 从一系列单词中预测缺失的单词
* 在一系列句子之后预测新的句子

让我们用这两种挑战来看看 BERT 的表现。

## 6.3 分词

由于神经网络是数值计算机，让我们将文本转换为数值 token。让我们加载 BERT 的[分词器](https://huggingface.co/docs/transformers/main_classes/tokenizer#tokenizer)：

In [2]:
import torch
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


BERT 的`分词器`可以一次性[编码](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.encode)多段文本。稍后我们将测试 BERT 的记忆力，先给它一些信息和一个关于信息的问题。您可以随时回到这里，尝试不同的句子组合。

In [3]:
text_1 = "I understand equations, both the simple and quadratical."
text_2 = "What kind of equations do I understand?"

# Tokenized input with special tokens around it (for BERT: [CLS] at the beginning and [SEP] at the end)
indexed_tokens = tokenizer.encode(text_1, text_2, add_special_tokens=True)
indexed_tokens

[101,
 146,
 2437,
 11838,
 117,
 1241,
 1103,
 3014,
 1105,
 186,
 18413,
 21961,
 1348,
 119,
 102,
 1327,
 1912,
 1104,
 11838,
 1202,
 146,
 2437,
 136,
 102]

如果我们计算 token 的数量，会发现句子中的 token 数量多于单词数。让我们来看看为什么会这样。可以用 [convert_ids_to_tokens](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.convert_ids_to_tokens) 来查看使用了哪些 token。

In [4]:
tokenizer.convert_ids_to_tokens([str(token) for token in indexed_tokens])

['[CLS]',
 'I',
 'understand',
 'equations',
 ',',
 'both',
 'the',
 'simple',
 'and',
 'q',
 '##uad',
 '##ratic',
 '##al',
 '.',
 '[SEP]',
 'What',
 'kind',
 'of',
 'equations',
 'do',
 'I',
 'understand',
 '?',
 '[SEP]']

索引列表比原始输入长的原因有两个：
1. `tokenizer` 添加了 `special_tokens` 来表示序列的开始（`[CLS]`）和句子之间的分隔（`[SEP]`）。
2. `tokenizer` 可以将一个词分解成多个部分。

从语言学角度来看，第二点很有趣。许多语言都有[词根](https://zh.wikipedia.org/wiki/%E8%AF%8D%E6%A0%B9)，或构成单词的组成部分。例如，“quadratic”这个词的词根是“quadr”，意思是“4”。BERT 并不是用语言定义词根，而是使用 [WordPiece](https://paperswithcode.com/method/wordpiece) 模型来找出如何拆分单词模式。我们今天使用的 BERT 模型有一个 `28996` 个 token 的词汇表。

我们可以直接[解码](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)编码过的文本。注意 `special_tokens` 已经被添加进去了。

In [5]:
tokenizer.decode(indexed_tokens)

'[CLS] I understand equations, both the simple and quadratical. [SEP] What kind of equations do I understand? [SEP]'

## 6.4 文本分段

为了使用 BERT 模型进行预测，它还需要一个 `segment_ids` 的列表。这是一个与我们 token 相同长度的向量，表示每个句子属于哪个段落。

由于我们的 `tokenizer` 添加了一些 `special_tokens`，我们可以使用这些特殊标记来找到段落。首先，让我们定义哪个索引对应哪个特殊标记。

In [6]:
cls_token = 101
sep_token = 102

接下来，我们可以创建一个 `for` 循环。我们将从 `segment_id` 设置为 `0` 开始，并且每当我们看到 [SEP] 标记时就增加 `segment_id`。为了确保效果，我们将在稍后将这些 `segment_ids` 和 `indexed_tokens` 作为张量输入模型。

In [7]:
def get_segment_ids(indexed_tokens):
    segment_ids = []
    segment_id = 0
    for token in indexed_tokens:
        if token == sep_token:
            segment_id += 1
        segment_ids.append(segment_id)
    segment_ids[-1] -= 1  # Last [SEP] is ignored
    return torch.tensor([segment_ids]), torch.tensor([indexed_tokens])

让我们测试一下。每个数字是否正确地对应第一句和第二句？

In [8]:
segments_tensors, tokens_tensor = get_segment_ids(indexed_tokens)
segments_tensors

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

## 6.4 文本掩码（Text Masking）

让我们先看看 BERT 对单词的处理。为了训练词嵌入，BERT 在一系列单词中掩掉一个单词。掩码用了一个特殊的标记：

In [9]:
tokenizer.mask_token

'[MASK]'

In [10]:
tokenizer.mask_token_id

103

让我们从之前的两个句子中选择位置索引 `5` 进行掩码。随时回到这里改变索引，看看结果会如何变化！

In [11]:
masked_index = 5

接下来，我们将应用掩码并验证它是否出现在句子序列中。

In [12]:
indexed_tokens[masked_index] = tokenizer.mask_token_id
tokens_tensor = torch.tensor([indexed_tokens])
tokenizer.decode(indexed_tokens)

'[CLS] I understand equations, [MASK] the simple and quadratical. [SEP] What kind of equations do I understand? [SEP]'

然后，我们将加载用于预测缺失单词的模型：`modelForMaskedLM`。

In [13]:
masked_lm_model = torch.hub.load('huggingface/pytorch-transformers', 'modelForMaskedLM', 'bert-base-cased')

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


就像使用其它 PyTorch 模块一样，我们可以检查其架构。

In [14]:
masked_lm_model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

您能找到标有 `word_embeddings` 的部分吗？这些是 BERT 为每个 token 学习到的嵌入。

In [15]:
embedding_table = next(masked_lm_model.bert.embeddings.word_embeddings.parameters())
embedding_table

Parameter containing:
tensor([[-0.0005, -0.0416,  0.0131,  ..., -0.0039, -0.0335,  0.0150],
        [ 0.0169, -0.0311,  0.0042,  ..., -0.0147, -0.0356, -0.0036],
        [-0.0006, -0.0267,  0.0080,  ..., -0.0100, -0.0331, -0.0165],
        ...,
        [-0.0064,  0.0166, -0.0204,  ..., -0.0418, -0.0492,  0.0042],
        [-0.0048, -0.0027, -0.0290,  ..., -0.0512,  0.0045, -0.0118],
        [ 0.0313, -0.0297, -0.0230,  ..., -0.0145, -0.0525,  0.0284]],
       requires_grad=True)

我们可以验证 BERT 词汇表中的 `28996` 个 token 都有一个大小为 `768` 的嵌入。

In [16]:
embedding_table.shape

torch.Size([28996, 768])

让我们测试一下模型！它能正确预测我们提供的句子中缺失的单词吗？我们将使用 [torch.no_grad](https://pytorch.org/docs/stable/generated/torch.no_grad.html) 来让 PyTorch 不计算梯度。

In [17]:
with torch.no_grad():
    predictions = masked_lm_model(tokens_tensor, token_type_ids=segments_tensors)
predictions

MaskedLMOutput(loss=None, logits=tensor([[[ -7.3832,  -7.2504,  -7.4539,  ...,  -6.0597,  -5.7928,  -6.2133],
         [ -6.7681,  -6.7896,  -6.8317,  ...,  -5.4655,  -5.4048,  -6.0683],
         [ -7.7323,  -7.9597,  -7.7348,  ...,  -5.7611,  -5.3566,  -4.3361],
         ...,
         [ -6.1213,  -6.3311,  -6.4144,  ...,  -5.8884,  -4.1157,  -3.1189],
         [-12.3216, -12.4479, -11.9787,  ..., -10.6539,  -8.7396, -11.0487],
         [-13.4115, -13.7876, -13.5183,  ..., -10.6359, -11.6582, -10.9009]]]), hidden_states=None, attentions=None)

这有点难读懂，让我们看一下 `shape`。

In [18]:
predictions[0].shape

torch.Size([1, 24, 28996])

`24` 指的是是 token 数，`28996` 是指对 BERT 词汇表中每个 token 的预测。我们想找到词汇表中所有 token 的最大值，可以用 [torch.argmax](https://pytorch.org/docs/stable/generated/torch.argmax.html)。

In [19]:
# Get the predicted token
predicted_index = torch.argmax(predictions[0][0], dim=1)[masked_index].item()
predicted_index

1241

让我们看看 token `1241` 对应的是什么：

In [20]:
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
predicted_token

'both'

您觉得怎么样？正确吗？

In [21]:
tokenizer.decode(indexed_tokens)

'[CLS] I understand equations, [MASK] the simple and quadratical. [SEP] What kind of equations do I understand? [SEP]'

## 6.5 问题与回答

虽然词掩码很有趣，但 BERT 是为更复杂的问题设计的，比如句子预测。它能通过 [Attention Transformer](https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf) 架构来完成这一任务。

在这一部分，我们将使用 BERT 的不同版本，它有自己的 tokenizer。让我们为我们的示例句子找到一组新的 token。

In [22]:
text_1 = "I understand equations, both the simple and quadratical."
text_2 = "What kind of equations do I understand?"

question_answering_tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-large-uncased-whole-word-masking-finetuned-squad')
indexed_tokens = question_answering_tokenizer.encode(text_1, text_2, add_special_tokens=True)
segments_tensors, tokens_tensor = get_segment_ids(indexed_tokens)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


接下来，让我们加载 `question_answering_model`。

In [23]:
question_answering_model = torch.hub.load('huggingface/pytorch-transformers', 'modelForQuestionAnswering', 'bert-large-uncased-whole-word-masking-finetuned-squad')

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


我们可以像在掩掉单词时一样输入 tokens 和 segments。

In [24]:
# Predict the start and end positions logits
with torch.no_grad():
    out = question_answering_model(tokens_tensor, token_type_ids=segments_tensors)
out

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-5.5943, -4.2960, -5.2682, -1.2511, -6.8350, -0.3992,  2.2274,  2.4654,
         -6.6066,  2.5014, -4.4613, -4.8040, -7.8383, -5.5944, -4.7833, -6.9730,
         -7.1477, -5.2967, -7.4825, -6.7737, -6.8806, -8.6612, -5.5944]]), end_logits=tensor([[-0.7409, -5.3478, -4.2317, -0.0275, -2.6293, -5.9589, -2.8828,  2.7770,
         -4.8512, -2.2092, -2.2413,  4.4412, -0.7181, -0.7411, -3.8988, -5.3865,
         -5.0452, -4.4974, -6.3098, -5.5938, -5.5562, -5.3034, -0.7412]]), hidden_states=None, attentions=None)

`question_answering_model` 和问答模型正在扫描我们的输入序列，以找到最能回答问题的子序列。数值越高，答案就越有可能是从这里开始的。

In [25]:
out.start_logits

tensor([[-5.5943, -4.2960, -5.2682, -1.2511, -6.8350, -0.3992,  2.2274,  2.4654,
         -6.6066,  2.5014, -4.4613, -4.8040, -7.8383, -5.5944, -4.7833, -6.9730,
         -7.1477, -5.2967, -7.4825, -6.7737, -6.8806, -8.6612, -5.5944]])

同样，`end_logits` 中的数越高，答案就越可能结束在那个 token 上。

In [26]:
out.end_logits

tensor([[-0.7409, -5.3478, -4.2317, -0.0275, -2.6293, -5.9589, -2.8828,  2.7770,
         -4.8512, -2.2092, -2.2413,  4.4412, -0.7181, -0.7411, -3.8988, -5.3865,
         -5.0452, -4.4974, -6.3098, -5.5938, -5.5562, -5.3034, -0.7412]])

然后我们可以用 [torch.argmax](https://pytorch.org/docs/stable/generated/torch.argmax.html) 来找到从开始到结束的 `answer_sequence`：

In [27]:
answer_sequence = indexed_tokens[torch.argmax(out.start_logits):torch.argmax(out.end_logits)+1]
answer_sequence

[17718, 23671, 2389]

最后，让我们[解码](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)这些 token，看看答案是否正确！

In [28]:
question_answering_tokenizer.convert_ids_to_tokens(answer_sequence)

['quad', '##ratic', '##al']

In [29]:
question_answering_tokenizer.decode(answer_sequence)

'quadratical'

## 6.7 总结

干得好！您成功地使用了大语言模型 (LLM) 从一系列句子中提取答案。尽管 BERT 在首次发布时是最先进的，但许多其它 LLM 自那以后也取得了突破。[build.nvidia.com](https://build.nvidia.com/explore/discover) 上托管了许多这样的模型，可以在浏览器中与它们互动。去探索一下，了解了解当今的最先进技术吧！

### 6.7.1 清理内存
继续进行后续内容之前，请执行以下单元清理 GPU 显存。

In [30]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### 6.7.2 下一步

恭喜您，您已经完成了所有课程学习目标！

作为最后的练习，请在评估中成功完成一个端到端的图像分类问题，完成后您将获得本课程的证书。